# Purpose

Develop a script that pulls from the approved.json file and tweets the memes/headlines to X. This is the final script in the three script pipeline: meme generation, selection, and publishing.

Imports

In [12]:
import datetime
import json
import os

from requests_oauthlib import OAuth1Session
from env import X_API_KEY, X_SECRET_KEY


Define the payload.

In [3]:
payload = {'text': 'Hello, World!'}

Receive a request token with the ability to write a tweet. The code is largely taken from the APIv2 documentation found [here](https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Manage-Tweets/create_tweet.py).

In [4]:
request_token_url = 'https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write'

oauth = OAuth1Session(X_API_KEY, client_secret = X_SECRET_KEY)

try:

    fetch_response = oauth.fetch_request_token(request_token_url)

except ValueError:

    print('Something went wrong..')    

Nothing errored. I have retrieved a request token that allows me to post a tweet. Now I need to extract the authorization keys present in the response.

In [5]:
resource_key = fetch_response.get('oauth_token')

resource_secret_key = fetch_response.get('oauth_token_secret')

Now that I have authorization keys, I need to receive authorization from X.

In [6]:
base_auth_url = 'https://api.twitter.com/oauth/authorize'

auth_url = oauth.authorization_url(base_auth_url)

print(f'Visit this URL to authorize the connection: {auth_url}')

verifier = input('Paste PIN here: ')

Visit this URL to authorize the connection: https://api.twitter.com/oauth/authorize?oauth_token=EuLy7QAAAAABrBo5AAABjSO33Eg


I visited the URL and authorized my application. Now I need to retrieve my access token.

In [7]:
access_token_url = 'https://api.twitter.com/oauth/access_token'

oauth = OAuth1Session(
    X_API_KEY,
    client_secret = X_SECRET_KEY,
    resource_owner_key = resource_key,
    resource_owner_secret = resource_secret_key,
    verifier = verifier
)

oauth_tokens = oauth.fetch_access_token(access_token_url)

access_token = oauth_tokens['oauth_token']
access_secret = oauth_tokens['oauth_token_secret']

I am able to retrieve an access token and a secret access token. I will use these tokens to authorize my request to post a tweet.

In [8]:
oauth = OAuth1Session(
    X_API_KEY,
    client_secret = X_SECRET_KEY,
    resource_owner_key = access_token,
    resource_owner_secret = access_secret
)

Now that I have a new oauth session with my consumer keys and access tokens, I can post a tweet.

In [9]:
response = oauth.post(
    'https://api.twitter.com/2/tweets',
    json=payload,
)

if response.status_code != 201:
    raise Exception(
        f'Received error {response.status_code} with message {response.text}.'
    )

The tweet is [live!](https://twitter.com/DodgyWaters/status/1748466969978830864) This is incredible. Can I make multiple tweets with the same authorization session?

In [10]:
new_payload = {'text': 'Is there anybody out there?'}

response = oauth.post(
    'https://api.twitter.com/2/tweets',
    json=new_payload,
)

if response.status_code != 201:
    raise Exception(
        f'Received error {response.status_code} with message {response.text}.'
    )

It's working! I don't have to redo authorization/authentication for successive posts. The resl question is.. will this break tomorrow?

Now for the next step: can I post an image with accompanying text?

In [14]:
today = datetime.date.today().strftime('%Y-%m-%d')

dbs_path = os.path.join(today, 'deepblue.jpg')

mixed_payload = {'text': 'Deep Blue Something.',
                 'image': dbs_path}

response = oauth.post(
    'https://api.twitter.com/2/tweets',
    json=mixed_payload,
)

if response.status_code != 201:
    raise Exception(
        f'Received error {response.status_code} with message {response.text}.'
    )

Exception: Received error 400 with message {"errors":[{"message":"$.image: is not defined in the schema and the schema does not allow additional properties"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}.

I hit a bit of a snag here. It appears I need to upload my image via API using the media endpoint, and then I have to reference the uploaded image to include it in my tweet.